# Ant Colony Optimization Problem
#### by Vico Wietstock, Jakob Litsch, Alexander Palatnik, Paul Ludwig Branzk and Friedrich Heitzer

In [1]:
import numpy as np
import timeit

### Input, Initialization (Friedrich Heitzer)

In [2]:
eingabe = input("Welche benchmark soll laufen? ")

#Reads the tsp file and creates a nested list out of it
liste = []
with open(eingabe + ".tsp") as f:
    for line in f:
        linelist = line.split(" ")
        linelist = linelist[:-1]
        linelist = list(map(int, linelist))
        liste.append(linelist)

# numpy array for distances between cities
distances = np.array(liste)

# number of cities
citycount = len(liste[0])
def p_init(citycount):
    # Initialization of the pheromones array
    pheromones = np.ones((citycount, citycount))
    for i in range(citycount):
        pheromones[i][i] = 0
    return pheromones

Welche benchmark soll laufen? 01manhattan


### Construct Solution (Alexander Palatnik, Jakob Litsch)

In [3]:
def constructsolution(pval, dval, citycount, a, b, antcount):
    solution = np.zeros((antcount, citycount-1), dtype = int)
    cities = np.zeros((citycount,), dtype = float)
    for ant in range(antcount):
        pos = 0
        chosencities = np.zeros((citycount,), dtype = int)
        chosencities[pos] = 0
        N = computeN(pos, pval, dval, a, b, chosencities)
        city = 0
        for city in range(citycount):
            if(city in chosencities):
                cities[city] = 0
            else:
                cities[city] = computeprob(pos, city, pval, dval, a, b, chosencities, N)
        #cities = computeprob(pos, city, pval, dval, a, b, chosencities, N, ph, d)
        #print(cities)
        #cities = np.nan_to_num(cities)
        for i in range(citycount-1):
            pos = np.random.choice(len(cities),1, p = cities)
            chosencities[i+1] = pos
            solution[ant, i] = pos
            N = computeN(pos, pval, dval, a, b, chosencities)
            for city in range(citycount):
                if(city in chosencities):
                    cities[city] = 0
                else:
                    cities[city] = computeprob(pos, city, pval, dval, a, b, chosencities, N)
           # cities = computeprob(pos, city, pval, dval, a, b, chosencities, N, ph, d)
            #cities = np.nan_to_num(cities)
    return solution

In [4]:
def computeprob(pos, city, pval, dval, a, b, chosencities, N):
    p = (1/N) * (pval[pos, city]**a/dval[pos, city]**b)
    return p

In [5]:
def computeN(pos, pval, dval, a, b, chosencities):
    N = 0
    for index in range(len(dval)):
        if index not in chosencities:
            N += pval[pos, index]**a/dval[pos, index]**b
   # ph = np.array([])
   # d = np.array([])
   # for i in range(len(dval)):
    #    if i  in chosencities:
   #         ph = np.append(ph, 0)
   #        d = np.append(d, dval[pos, i]**b)
   #     else:
   #         ph = np.append(ph, pval[pos, i]**a)
  #          d = np.append(d, dval[pos, i]**b)
   # divided = np.array([])
   # divided = ph/d
   # divided[0] = 0
   # N = np.sum(divided)
   # print(N)        
    return N

### Evaluation, Evaporization, Intensification (Vico Wietstock, Friedrich Heitzer)

In [6]:
# returns distance of the given solution/ant 
def get_distance(distances, ant):
    
    distance = 0        
    # iterates through cities in the ant and sums up distances 
    for city, nextcity in zip(ant[:-1], ant[1:]):
        distance += distances[city][nextcity]
        
    return distance

In [7]:
# returns the best solution/ant
def get_best(ants, distances):
    values = []
    # Interates through ant population
    for ant in ants:
        distance = distances[0][ant[0]]
        
        # double iterates through the cities in the seperate solutions aka ants
        
        for city, nextcity in zip(ant[:-1], ant[1:]):
            distance += distances[city][nextcity]
        values.append(distance)
    # returns the best solution aka ant
    return ants[values.index(min(values))]

In [8]:
# Evaporization 
def evaporize(pheromones, evaporize_val):
    # evaporize all pheromone values
    pheromones *= evaporize_val
    
    return pheromones

In [9]:
# Intensification
def intensify(pheromones, intens_val, ants, distances):
    # get best solution 
    best_ant = get_best(ants, distances)
    
    # intensify values from the best solution
    for city, nextcity in zip(best_ant[:-1], best_ant[1:]):
        pheromones[city][nextcity] += intens_val
        
    return pheromones, best_ant

### Main (Ludwig Branzk, Vico Wietstock)

In [10]:
def main(distances, citycount, antcount, evaporize_val, intens_val, alpha, beta):
    pheromones = p_init(citycount)
    
    population = constructsolution(pheromones, distances, citycount, alpha, beta, antcount)
    pheromones = evaporize(pheromones, evaporize_val)
    pheromones, best_solution = intensify(pheromones, intens_val, population, distances)
    
    best_distance = get_distance(distances, best_solution)
    
    counter = 0
    
    while counter < 10:
        population = constructsolution(pheromones, distances, citycount, alpha, beta, antcount)
        pheromones = evaporize(pheromones, evaporize_val)
        pheromones, best_ant = intensify(pheromones, intens_val, population, distances)
        
        distance = get_distance(distances, best_ant)
        
        if distance < best_distance:
            best_solution = best_ant
            best_distance = distance
            counter = 0
        else:
            counter += 1
        print("Counter: ", counter)
        print("Best distance: ", best_distance)
    return best_distance

In [19]:
antcount = 20
evaporize_val = 0.9
intens_val = 0.1
alpha = 1.5
beta = 3

antcount_values = [24]

evap_values = [0.9, 0.7]

intens_values = [0.1, 0.3]

alpha_values = [1, 2]

beta_values = [1, 2]

#Your statements here
bEsTe_DiSt = []
models = []
times = []
overall_times = []
overall_dist = []
#for antcount in antcount_values:
 #   for evap in evap_values:
   #     for intens in intens_values:
      #      for alpha in alpha_values:
        #        for beta in beta_values:
                    
                    
         #           for _ in range(5):
         #               print("Next Running Model: ",antcount, evap, intens, alpha, beta)

         #               start = timeit.default_timer()
         #               bEsTe_DiSt.append(main(distances, citycount, antcount, evap, intens, alpha, beta))
                        
        #                stop = timeit.default_timer()
         #               print("The past model took ", (stop - start)/60, "s")
         #               print(" ")
      #                  times.append((stop - start)/60)
                
      #          models.append([antcount, evap, intens, alpha, beta])
     #          overall_dist.append(bEsTe_DiSt)
      #          overall_times.append(times)
#for i in range(10): 
start = timeit.default_timer()
bEsTe_DiSt.append(main(distances, citycount, antcount, evaporize_val, intens_val, alpha, beta))
stop = timeit.default_timer()
times.append((stop - start)/60)    


Counter:  1
Best distance:  6392
Counter:  0
Best distance:  6376
Counter:  0
Best distance:  6336
Counter:  1
Best distance:  6336
Counter:  0
Best distance:  6015
Counter:  1
Best distance:  6015
Counter:  2
Best distance:  6015
Counter:  3
Best distance:  6015
Counter:  4
Best distance:  6015
Counter:  5
Best distance:  6015
Counter:  0
Best distance:  5790
Counter:  0
Best distance:  5299
Counter:  1
Best distance:  5299
Counter:  2
Best distance:  5299
Counter:  3
Best distance:  5299
Counter:  0
Best distance:  5254
Counter:  1
Best distance:  5254
Counter:  2
Best distance:  5254
Counter:  3
Best distance:  5254
Counter:  4
Best distance:  5254
Counter:  5
Best distance:  5254
Counter:  0
Best distance:  5213
Counter:  1
Best distance:  5213
Counter:  0
Best distance:  4855
Counter:  1
Best distance:  4855
Counter:  0
Best distance:  4767
Counter:  1
Best distance:  4767
Counter:  2
Best distance:  4767
Counter:  0
Best distance:  4584
Counter:  1
Best distance:  4584
Counter:  